# Customer review topic understanding using Snowflake Cortex
Understanding customer feedback is critical for businesses, but analyzing large volumes of unstructured text can be challenging. In this notebook, you'll use Cortex AISQL to systematically getting insights from unstructured customer feedback.

### Context
*Tasty Bytes* is a global e-commerce company selling different merchandise. They collect customer reviews to gain insights into the feedback on the product they provide.

In this notebook, we will leverage multiple AISQL functions to answer different use case questions upon customer reviews.

This enhanced implementation transforms basic customer review analysis into a comprehensive **Customer Intelligence Platform** that drives strategic business decisions, predicts customer behavior, and automates customer experience optimization.


- 🔮 **Predictive Analytics**: Customer churn prediction
- 🎯 **Customer Segmentation**: Customer Intelligence & Segmentation and AI-powered Customer health scoring
- 🤖 **Automated Workflows**: Intelligent response generation and alert systems


##

In [ ]:
import streamlit as st
import pandas as pd
import altair as alt
from snowflake.snowpark.context import get_active_session
session=get_active_session()

session.query_tag = {"origin":"sf_sit-is", "name":"AISQL-HOL", "version":{"major":1, "minor":0}, "attributes":{"is_quickstart":3, "source":"notebook"}}
print(session)

In [ ]:
-- Set the correct schema context for accessing data
-- Data assets are in the default schema, not the notebooks schema
USE SCHEMA DEFAULT_SCHEMA;

Let us quickly review the customer data

In [ ]:
-- Quick review of the table
SELECT * FROM customer_data limit 10;

Let us quickly review the product catalog data

In [ ]:
-- Quick review of the table
SELECT *
FROM product_catalog limit 10
;

Let us quickly review the product reviews dataset

In [ ]:
-- Quick review of the table
SELECT *
FROM product_reviews limit 10
;

## Correlate sentiment with ratings

As a first step, let's perform a quick sanity check. We'll use the `SNOWFLAKE.CORTEX.SENTIMENT` function to score the sentiment of each review. We can then check its correlation with the user-provided star rating to see if they align.

In [ ]:
WITH EXTRACTED_SENTIMENT AS (
    SELECT 
        RATING,
        SNOWFLAKE.CORTEX.SENTIMENT(REVIEW_TEXT) AS SENTIMENT
    FROM PRODUCT_REVIEWS
)
SELECT CORR(SENTIMENT, RATING) AS SENTIMENT_RATING_CORRELATION
FROM EXTRACTED_SENTIMENT;

## Find top issues in a category

Now, let's dig deeper. Suppose you want to know what the biggest complaints are for 'Electronics'. You can focus on the ones with negative sentiments, and use `AI_AGG` to analyze all relevant reviews and aggregate the common themes into a single summary.

In [ ]:
query='''SELECT 
  AI_AGG(
    REVIEW_TEXT, 
    'List the top 3 product issues mentioned across these reviews.'
  ) AS TOP_ISSUES
FROM PRODUCT_REVIEWS pr
JOIN PRODUCT_CATALOG pc 
  ON pr.product_id = pc.product_id
WHERE pc.category = 'Electronics'
  AND SNOWFLAKE.CORTEX.SENTIMENT(REVIEW_TEXT) < 0'''
top_issues_df = session.sql(query).to_pandas()


In [ ]:
st.markdown("### 🔍 Top 3 Product Issues in Negative Reviews")
st.write(top_issues_df["TOP_ISSUES"].iloc[0])  

## Identify the most common issue in Clothing category.

To answer this question, we start with filtering to Clothing category  Another way to identify comments that mentioned product issue is to leverage our latest [AI_FILTER](https://docs.snowflake.com/sql-reference/functions/ai_filter) to conduct filtering using natural language.

The next step we use the [AI_AGG](https://docs.snowflake.com/sql-reference/functions/ai_agg) function to get a list of all product issues mentioned.

In [ ]:

-- Create temporary table on the filtered result to be re-used in next step analytics.

CREATE OR REPLACE TEMP TABLE filtered_product_reviews AS
SELECT *
FROM product_reviews
WHERE AI_FILTER(PROMPT('This review mentions a product issue or complaint: {0}', review_text));

SELECT
  AI_AGG(
    review_text, 
    'Analyze these clothing product reviews and provide a comprehensive list of all product issues mentioned. Format your response as a bulleted list of issues with their approximate frequency in percentage.'
  ) as clothing_issues
FROM filtered_product_reviews pr
JOIN product_catalog pc ON pr.product_id = pc.product_id
WHERE pc.category = 'Clothing'

In [ ]:
# to view the result
df = COMMON_ISSUE_SQL.to_pandas()
print(df['CLOTHING_ISSUES'].iloc[0])

## 🧠 Advanced Customer Intelligence & Segmentation

### AI-Powered Customer Persona Analysis
Beyond basic demographics, we'll use AISQL to identify sophisticated customer personas based on behavior patterns.


In [ ]:

-- Advanced Customer Intelligence Analysis
CREATE OR REPLACE VIEW CUSTOMER_INTELLIGENCE AS
WITH customer_behavior_analysis AS (
    SELECT 
        c.customer_id,
        c.customer_segment,
        c.lifetime_value,
        c.age_range,
        c.preferred_category,
        c.previous_purchases,
        
        SNOWFLAKE.CORTEX.CLASSIFY_TEXT(
            CONCAT('Customer Profile: LTV=$', c.lifetime_value, 
                   ', Purchases=', c.previous_purchases, 
                   ', Age=', c.age_range, 
                   ', Preferred Category=', c.preferred_category,
                   ', Segment=', c.customer_segment),
            ['high_value_advocate', 'price_sensitive_shopper', 'quality_focused_buyer', 
             'trend_follower', 'occasional_purchaser', 'at_risk_churner']
        ) as customer_persona,
        -- Sentiment-based customer satisfaction scoring
        COALESCE(AVG(SNOWFLAKE.CORTEX.SENTIMENT(pr.review_text)), 0) as avg_sentiment_score,
        COUNT(pr.review_id) as review_count,
        COALESCE(AVG(pr.rating), 0) as avg_rating,
        MAX(pr.review_date) as last_review_date,
        -- Calculate customer engagement level
        CASE 
            WHEN COUNT(pr.review_id) >= 5 AND AVG(pr.rating) >= 4 THEN 'Highly Engaged'
            WHEN COUNT(pr.review_id) >= 3 AND AVG(pr.rating) >= 3 THEN 'Moderately Engaged'
            WHEN COUNT(pr.review_id) >= 1 THEN 'Low Engagement'
            ELSE 'No Reviews'
        END as engagement_level
    FROM customer_data c
    LEFT JOIN product_reviews pr ON c.customer_id = pr.customer_id
    GROUP BY c.customer_id, c.customer_segment, c.lifetime_value, 
             c.age_range, c.preferred_category, c.previous_purchases
)
SELECT 
    customer_id,
    customer_segment,
    -- Fixed JSON parsing for customer persona
    COALESCE(TRY_PARSE_JSON(customer_persona):label::string, 'quality_focused_buyer') as ai_persona,
    engagement_level,
    lifetime_value,
    ROUND(avg_sentiment_score, 3) as sentiment_score,
    review_count,
    ROUND(avg_rating, 2) as avg_rating,
    last_review_date,
    -- Fixed days calculation to handle NULLs
    CASE 
        WHEN last_review_date IS NOT NULL THEN DATEDIFF('day', last_review_date, CURRENT_DATE()) 
        ELSE NULL 
    END as days_since_last_review
FROM customer_behavior_analysis;





In [ ]:

query = f'''SELECT 
    customer_id,
    customer_segment,
    ai_persona,
    engagement_level,
    lifetime_value,
    sentiment_score,
    review_count,
    avg_rating
FROM CUSTOMER_INTELLIGENCE 
ORDER BY lifetime_value DESC 
LIMIT 10'''
df= session.sql(query).to_pandas()

col1, col2, col3 = st.columns(3)
col1.metric("👥 Total Customers", len(df))
col2.metric("🧠 With Persona", df["AI_PERSONA"].notna().sum())




# --- Chart 1: LTV vs Engagement ---
st.subheader("💰 Lifetime Value by Engagement Level")

chart2 = alt.Chart(df).mark_circle(size=60).encode(
    x="ENGAGEMENT_LEVEL:N",
    y="LIFETIME_VALUE:Q",
    color="AI_PERSONA:N",
    tooltip=["CUSTOMER_ID", "LIFETIME_VALUE", "AI_PERSONA", "ENGAGEMENT_LEVEL"]
).properties(width=600, height=300)

st.altair_chart(chart2, use_container_width=True)

# --- Table: Top Customers ---
st.subheader("🏅 Top 10 Customers by Lifetime Value")

top_customers = df.sort_values("LIFETIME_VALUE", ascending=False).head(10)
st.dataframe(top_customers[[
    "CUSTOMER_ID", "CUSTOMER_SEGMENT", "AI_PERSONA", "ENGAGEMENT_LEVEL",
    "LIFETIME_VALUE", "SENTIMENT_SCORE", "REVIEW_COUNT", "AVG_RATING"
]])


## 🧠 Predictive Customer Health Scoring
Identify customers at risk of churn and generate personalized retention strategies.


In [ ]:
query= f"""
WITH customer_health_scoring AS (
    SELECT 
        ci.*,
        -- AI-powered churn risk assessment
        AI_CLASSIFY(
            CONCAT('Customer Analysis: Sentiment=', sentiment_score, 
                   ', Days Since Last Review=', days_since_last_review,
                   ', Rating Trend=', avg_rating,
                   ', Engagement=', engagement_level,
                   ', LTV=$', lifetime_value),
            ['high_churn_risk', 'moderate_churn_risk', 'low_churn_risk', 'growth_opportunity']
        ) as churn_risk_assessment,

        -- Calculate customer health score (0-100)
        CASE 
            WHEN sentiment_score >= 0.5 AND avg_rating >= 4 AND days_since_last_review <= 30 THEN 90 + (sentiment_score * 10)
            WHEN sentiment_score >= 0.2 AND avg_rating >= 3.5 AND days_since_last_review <= 60 THEN 70 + (sentiment_score * 20)
            WHEN sentiment_score >= 0 AND avg_rating >= 3 THEN 50 + (sentiment_score * 30)
            WHEN sentiment_score >= -0.3 THEN 30 + ((sentiment_score + 0.3) * 40)
            ELSE 10 + ((sentiment_score + 1) * 20)
        END as customer_health_score
    FROM CUSTOMER_INTELLIGENCE ci
    WHERE review_count > 0
)

SELECT 
    customer_id,
    ai_persona,
    customer_segment,
    churn_risk_assessment:labels[0]::text as churn_risk,
    ROUND(customer_health_score, 0) as health_score,
    lifetime_value,
    sentiment_score,
    engagement_level,

    
    AI_COMPLETE(
        'llama4-maverick',
        'Customer Profile: ' || ai_persona || ' (' || customer_segment || ') ' ||
        'with $' || lifetime_value || ' LTV, ' || ROUND(customer_health_score, 0) || '/100 health score, ' ||
        'and ' || churn_risk_assessment:labels[0]::text || '. Generate  in 1 line an actionable retention strategy.'
    ) as retention_strategy

FROM customer_health_scoring
WHERE churn_risk_assessment:labels[0]::text IN ('high_churn_risk', 'moderate_churn_risk')
   OR customer_health_score < 60
ORDER BY 
    CASE churn_risk_assessment:labels[0]::text 
        WHEN 'high_churn_risk' THEN 1 
        WHEN 'moderate_churn_risk' THEN 2 
        ELSE 3 
    END,
    lifetime_value DESC
LIMIT 10"""


In [ ]:
df = session.sql(query).to_pandas()
df.columns = [col.lower() for col in df.columns]

# Check if data is returned
if df.empty:
    st.warning("No data found for customers with moderate or high churn risk.")
else:
    st.subheader("Top 10 Customers Needing Attention")
    st.dataframe(df)

    # Bar chart: Customer Health Score vs. Lifetime Value
    chart1 = alt.Chart(df).mark_bar().encode(
        x=alt.X("customer_id:N", title="Customer ID", sort="-y"),
        y=alt.Y("health_score:Q", title="Health Score"),
        color=alt.Color("churn_risk:N", title="Churn Risk"),
        tooltip=["customer_id", "health_score", "churn_risk", "lifetime_value"]
    ).properties(
        width=600,
        height=300,
        title="Customer Health Score by ID"
    )

    # Bubble chart: Lifetime Value vs. Sentiment with Churn Risk
    chart2 = alt.Chart(df).mark_circle(size=100).encode(
        x=alt.X("lifetime_value:Q", title="Lifetime Value ($)"),
        y=alt.Y("sentiment_score:Q", title="Sentiment Score"),
        color=alt.Color("churn_risk:N", title="Churn Risk"),
        size="health_score:Q",
        tooltip=["customer_id", "lifetime_value", "sentiment_score", "churn_risk"]
    ).properties(
        width=600,
        height=300,
        title="Customer Sentiment vs. Lifetime Value"
    )

    st.altair_chart(chart1, use_container_width=True)
    st.altair_chart(chart2, use_container_width=True)

    # Show strategies
    st.subheader("Retention Strategies (AI-Generated)")
    for i, row in df.iterrows():
        st.markdown(f"**{row['customer_id']}** ({row['churn_risk']}): {row['retention_strategy']}")

## 🚨 Alert System

### Intelligent Early Warning System
Real-time detection of critical customer experience issues with automated alert generation.


In [ ]:
-- Customer Experience Alert System
CREATE OR REPLACE VIEW CUSTOMER_EXPERIENCE_ALERTS AS
WITH recent_review_analysis AS (
    SELECT 
        pr.product_id,
        pc.product_name,
        pc.category,
        pc.manufacturer,
        pr.review_text,
        pr.rating,
        pr.review_date,
        SNOWFLAKE.CORTEX.SENTIMENT(pr.review_text) as sentiment_score,

        -- ✅ Correct AI_FILTER prompts using PROMPT()
        AI_FILTER(PROMPT('This review mentions safety concerns, defects, or serious quality issues: {0}', pr.review_text)) as has_safety_concern,
        AI_FILTER(PROMPT('This review expresses intent to return the product or request a refund: {0}', pr.review_text)) as has_return_intent,
        AI_FILTER(PROMPT('This review mentions competitor products or switching to competitors: {0}', pr.review_text)) as has_competitive_mention

    FROM product_reviews pr
    JOIN product_catalog pc ON pr.product_id = pc.product_id
    
)
,
alert_conditions AS (
    SELECT 
        product_id,
        product_name,
        category,
        manufacturer,
        COUNT(*) as recent_review_count,
        ROUND(AVG(sentiment_score), 3) as avg_sentiment,
        ROUND(AVG(rating), 2) as avg_rating,
        SUM(CASE WHEN has_safety_concern THEN 1 ELSE 0 END) as safety_concern_count,
        SUM(CASE WHEN has_return_intent THEN 1 ELSE 0 END) as return_intent_count,
        SUM(CASE WHEN has_competitive_mention THEN 1 ELSE 0 END) as competitive_mention_count,
        -- Determine alert type and severity
        CASE 
            WHEN SUM(CASE WHEN has_safety_concern THEN 1 ELSE 0 END) >= 2 THEN 'CRITICAL_SAFETY_ALERT'
            WHEN AVG(sentiment_score) < -0.6 AND COUNT(*) >= 5 THEN 'CRITICAL_SENTIMENT_DROP'
            WHEN AVG(rating) < 2.0 AND COUNT(*) >= 3 THEN 'CRITICAL_RATING_DROP'
            WHEN SUM(CASE WHEN has_return_intent THEN 1 ELSE 0 END) >= 3 THEN 'HIGH_RETURN_RISK'
            WHEN SUM(CASE WHEN has_competitive_mention THEN 1 ELSE 0 END) >= 3 THEN 'COMPETITIVE_THREAT'
            WHEN AVG(sentiment_score) < -0.3 AND COUNT(*) >= 3 THEN 'MODERATE_CONCERN'
            ELSE 'NORMAL'
        END as alert_type
        
    FROM recent_review_analysis
    GROUP BY product_id, product_name, category, manufacturer
)
SELECT 
    product_id,
    product_name,
    category,
    manufacturer,
    alert_type,
    recent_review_count,
    avg_sentiment,
    avg_rating,
    safety_concern_count,
    return_intent_count,
    competitive_mention_count,
    CURRENT_TIMESTAMP() as alert_generated_at,
    -- Generate AI-powered alert summary and recommended actions
     AI_COMPLETE(
        'llama4-maverick',
        'ALERT: ' || alert_type || ' for ' || product_name || ' (' || category || '). ' ||
        'Recent metrics: ' || recent_review_count || ' reviews, ' || avg_sentiment || ' sentiment, ' ||
        avg_rating || ' rating. Safety concerns: ' || safety_concern_count || '. ' ||
        'Provide immediate action plan with 1) Urgent steps, 2) Investigation priorities, 3) Communication strategy.'
    ) as action_plan
FROM alert_conditions WHERE
alert_type != 'NORMAL'
ORDER BY avg_sentiment ASC;

-- Display current active alerts
SELECT 
    alert_type,
    product_name,
    category,
    avg_sentiment,
    avg_rating,
    action_plan
FROM CUSTOMER_EXPERIENCE_ALERTS
LIMIT 5;


## 💡Intelligent Response Generation & Automation

### Advanced Personalized Response Engine
Generate contextually-aware, segment-specific customer service responses with follow-up recommendations.


## Generate responses to customer complaints

Finally, let's close the loop. You can use `AI_COMPLETE` to help your support team draft empathetic and relevant responses to negative reviews, improving customer satisfaction at scale.

In [ ]:
WITH clothing_issue_reviews AS (
  SELECT 
    pr.review_id,
    pr.review_text
  FROM filtered_product_reviews pr
  JOIN product_catalog pc ON pr.product_id = pc.product_id
  WHERE pc.category = 'Clothing'
)
SELECT 
    review_id,
    review_text,
    AI_COMPLETE('llama4-maverick', 'Please help me draft a concise response to the customer complaints below. Please only include the draft and nothing else: ' || review_text) as response
  FROM clothing_issue_reviews


## Key Takeaways

* **Customer Intelligence Platform** that drives strategic business decisions and creates measurable value across the organization. 

This enhanced implementation transformed a basic customer review analysis into a comprehensive **Customer Intelligence Platform** that drives strategic business decisions, predicts customer behavior, and automates customer experience optimization.


- 🔮 **Predictive Analytics**: Customer churn prediction
- 🎯 **Customer Segmentation**: Customer Intelligence & Segmentation and AI-powered Customer health scoring
- 🤖 **Automated Workflows**: Intelligent response generation and alert systems



##

## Additional Resources

* [Documentation: Cortex AI SQL Functions](https://docs.snowflake.com/en/user-guide/snowflake-cortex/aisql)